In [ ]:
import os
import sys
import math
import argparse
import numpy as np
from collections import Counter

# Keras imports
from keras import layers
from keras import backend as K
from tensorflow.keras.models import Model
from keras.layers import Input, LSTM, Dense, Permute, Reshape, multiply, concatenate, RepeatVector

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.models import train_model_from_generators

# IPython imports
from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
with open('/home/sam/europarl-v7.es-en.en', 'r', encoding='utf-8') as english_file:
    with open('/home/sam/europarl-v7.es-en.es', 'r', encoding='utf-8') as spanish_file:
        english = english_file.read()
        spanish = spanish_file.read()

In [ ]:
start_char = '\t'
end_char = '\n'
english_lines = english.split('\n')
spanish_lines = [start_char + l + end_char for l in spanish.split('\n')]

In [ ]:
english_chars = sorted(list(set(english)))
print('total english chars:', len(english_chars))
spanish_chars = sorted(list(set(spanish + start_char + end_char)))
print('total spanish chars:', len(spanish_chars))
chars = sorted(list(set(english_chars + spanish_chars)))
print('total chars:', len(chars))

In [ ]:
max_english = 100 # max(map(len, english_lines))//2
max_spanish = 100 # max(map(len, spanish_lines))//2
max_pairs = 15000

In [ ]:
english_token_index = {char: i for i, char in enumerate(english_chars)}
spanish_token_index = {char: i for i, char in enumerate(spanish_chars)}
english_index_tokens = {i: char for i, char in enumerate(english_chars)}
spanish_index_tokens = {i: char for i, char in enumerate(spanish_chars)}
encoder_input_data = np.zeros((max_pairs, max_english, len(english_chars)), dtype='float32')
decoder_input_data = np.zeros((max_pairs, max_spanish, len(spanish_chars)), dtype='float32')
decoder_target_data = np.zeros((max_pairs, max_spanish, len(spanish_chars)), dtype='float32')

for i, (english_text, spanish_text) in enumerate(zip(english_lines, spanish_lines)):
    for t, char in enumerate(english_text):
        if t == max_english:
            break
        encoder_input_data[i, t, english_token_index[char]] = 1.
    for t, char in enumerate(spanish_text):
        if t == max_spanish:
            break
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, spanish_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep and will not include the start character.
            decoder_target_data[i, t - 1, spanish_token_index[char]] = 1.
    if i == max_pairs-1:
        break

In [ ]:
latent_dim = 256
batch_size = 64
epochs = 6

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, len(english_chars)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, len(spanish_chars)))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(spanish_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
# Save model
model.save('s2s.h5')

In [ ]:

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~ Sample with Temperature ~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def sample(preds, temperature=1.0):
	# helper function to sample an index from a probability array
	preds = np.asarray(preds).astype('float64')
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
	return np.argmax(probas)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(spanish_chars)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, spanish_token_index[start_char]] = 1.

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = spanish_index_tokens[sampled_token_index]
        decoded_sentence += sampled_char
        #print('sampled char is:', sampled_token_index, sampled_char)
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == end_char or len(decoded_sentence) > max_spanish):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(spanish_chars)))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', english_lines[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Actual sentence:', spanish_lines[seq_index])

In [ ]:
def attention_3d_block(inputs, time_steps):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, time_steps))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(time_steps, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

In [ ]:
latent_dim = 256
batch_size = 64
epochs = 96
from keras.layers import RepeatVector, concatenate
# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_english, len(english_chars)))
encoder_out = LSTM(latent_dim)(encoder_inputs)
encoder_repeat = RepeatVector(max_spanish)(encoder_out)
attended_encoding = attention_3d_block(encoder_repeat, max_spanish)

burn_in = Input(shape=(max_spanish, len(spanish_chars)))
lstm_in = layers.concatenate([burn_in, attended_encoding], name='concat_spanish_and_attended')
decoder_lstm = LSTM(latent_dim, return_sequences=True)
decoder_outputs = decoder_lstm(lstm_in)
decoder_outputs = Dense(len(spanish_chars), activation='softmax')(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, burn_in], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


In [ ]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
# Save model
model.save('s2s_attention_96.h5')

In [ ]:
# Define sampling models
attn_encoder_model = Model(encoder_inputs, attended_encoding)
attn_encoder_model.summary()


In [ ]:
def decode_attended_sequence(input_seq):
    # Encode the input as state vectors.
    attended_np = attn_encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    burn_in_np = np.zeros((1, max_spanish, len(spanish_chars)))
    # Populate the first character of target sequence with the start character.
    burn_in_np[0, 0, spanish_token_index[start_char]] = 1.

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    char_idx = 0
    while not stop_condition:
        char_idx += 1
        output_tokens = model.predict([input_seq, burn_in_np])
        # Sample a token
        sampled_token_index =  sample(output_tokens[0, char_idx-1, :], 1.1)
        sampled_char = spanish_index_tokens[sampled_token_index]
        decoded_sentence += sampled_char

        # Update the target sequence (of length 1).
        burn_in_np[0, char_idx, sampled_token_index] = 1.
        
        
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == end_char or len(decoded_sentence) > max_spanish) or char_idx == max_spanish-1:
            stop_condition = True

    return decoded_sentence

In [ ]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_attended_sequence(input_seq)
    print('-')
    print('Input sentence:', english_lines[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Actual sentence:', spanish_lines[seq_index])